In [147]:
import numpy as np
import cv2 

In [148]:

# Function to compute histogram of an image
def compute_histogram(image):
    hist = np.zeros(256, dtype=int)
    for pixel in image.flatten():
        hist[pixel] += 1
    return hist

# Function to normalize the histogram
def normalize_histogram(hist, total_pixels):
    return hist / total_pixels

# Function to compute cumulative sums and means
def compute_cumulative_sums_and_means(hist):
    cum_sum = np.cumsum(hist)
    cum_mean = np.cumsum(np.arange(256) * hist)
    return cum_sum, cum_mean


In [149]:

def refine_foreground_segmentation(image, thresh, optimal_threshold):
    """
    Refines the thresholding using Otsu's method on the foreground pixels defined by thresh.
    
    Parameters:
        image (2D numpy array): Grayscale image or feature map.
        thresh (2D numpy array): Binary mask obtained from initial thresholding.
        optimal_threshold (int): Initial threshold value from Otsu's method.
        
    Returns:
        refined_mask (2D numpy array): Refined binary mask after applying Otsu algorithm iteratively.
    """
    # Step 1: Extract the preliminary foreground using the initial threshold
    foreground_pixels = image[thresh == 255]
    
    # Step 2: Apply Otsu's threshold to the foreground pixels to refine the thresholding
    if len(foreground_pixels) > 0:
        refined_threshold = otsu_threshold(foreground_pixels)[0]
    else:
        refined_threshold = optimal_threshold  # Use the original threshold if no foreground is detected
    
    # Step 3: Create the final refined mask
    # Use the refined threshold on the original image, but only where the initial foreground was detected
    refined_mask = np.zeros_like(image, dtype=np.uint8)
    refined_mask[(image > refined_threshold) & (thresh == 255)] = 255
    
    return refined_mask


def iterative_otsu(image_channel, iterations):
    """
    Apply the Otsu algorithm iteratively on an image channel for a specified number of iterations.
    
    Parameters:
        image_channel (2D numpy array): Grayscale image channel.
        iterations (int): Number of iterations for refining the Otsu threshold.
        
    Returns:
        final_mask (2D numpy array): Final binary mask after specified iterations.
    """
    # Step 1: Apply the initial Otsu threshold
    optimal_threshold, initial_mask = otsu_threshold(image_channel)
    
    # Step 2: Refine the mask iteratively
    refined_mask = initial_mask
    for i in range(iterations):
        refined_mask = refine_foreground_segmentation(image_channel, refined_mask, optimal_threshold)
    
    return refined_mask




# def iterative_otsu_for_texture_maps(feature_maps, iterations_list):
#     """
#     Apply iterative Otsu's thresholding to each feature map using a distinct number of iterations.
    
#     Parameters:
#         feature_maps (list): List of 2D feature maps.
#         iterations_list (list): Number of iterations for each feature map.
        
#     Returns:
#         refined_masks (list): List of refined binary masks after iterative Otsu.
#     """
#     refined_masks = []
    
#     for feature_map, iterations in zip(feature_maps, iterations_list):
#         initial_threshold, initial_mask = otsu_threshold(feature_map)
#         refined_mask = initial_mask
        
#         # Perform the iterative refinement
#         for _ in range(iterations):
#             refined_mask = refine_foreground_segmentation(feature_map, refined_mask, initial_threshold)
        
#         refined_masks.append(refined_mask)
    
#     return refined_masks


def combine_masks_with_bitwise_and(masks):
    """
    Combine binary masks using a logical AND operation.
    
    Parameters:
        masks (list): List of binary masks (2D arrays).
        
    Returns:
        combined_mask (2D numpy array): Combined binary mask.
    """
    # Start with the first mask
    combined_mask = masks[0]
    
    # Apply bitwise AND with subsequent masks
    for mask in masks[1:]:
        combined_mask = np.bitwise_and(combined_mask, mask)
    
    return combined_mask

# Function to find optimal threshold using Otsu's method
def otsu_threshold(image):
    # Step 1: Compute histogram
    hist = compute_histogram(image)
    
    # Step 2: Normalize the histogram
    total_pixels = image.size
    prob_hist = normalize_histogram(hist, total_pixels)
    
    # Step 3: Compute cumulative sums and means
    cum_sum, cum_mean = compute_cumulative_sums_and_means(prob_hist)
    
    # Step 4: Compute global mean
    global_mean = cum_mean[-1]
    
    # Step 5: Initialize variables to find optimal threshold
    max_variance = -1
    optimal_threshold = 0
    
    for t in range(256):
        # Probabilities of two classes (background and foreground)
        w0 = cum_sum[t]  # Background class
        w1 = 1 - w0      # Foreground class
        
        if w0 == 0 or w1 == 0:
            continue  # Avoid division by zero
        
        # Means of the two classes
        mean0 = cum_mean[t] / w0
        mean1 = (cum_mean[-1] - cum_mean[t]) / w1
        
        # Step 6: Compute between-class variance
        between_class_variance = w0 * w1 * (mean0 - mean1) ** 2
        
        # Step 7: Update the optimal threshold if the variance is maximal
        if between_class_variance > max_variance:
            max_variance = between_class_variance
            optimal_threshold = t
    
    # Step 8: Threshold the image using the optimal threshold
    thresh = np.where(image > optimal_threshold, 255, 0).astype(np.uint8)
    
    return optimal_threshold, thresh





In [150]:
def compute_texture_features(image, N):
    """
    Compute the intensity variance within an N x N window around each pixel.
    
    Parameters:
        image (2D numpy array): Grayscale image.
        N (int): Size of the sliding window (e.g., 3, 5, or 7).
        
    Returns:
        texture_map (2D numpy array): Feature map with intensity variance.
    """
    pad = N // 2  # Padding size for border handling
    padded_image = np.pad(image, pad, mode='constant', constant_values=0)
    texture_map = np.zeros_like(image, dtype=np.float32)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            window = padded_image[i:i + N, j:j + N]
            mean_intensity = np.mean(window)
            variance = np.mean((window - mean_intensity) ** 2)
            texture_map[i, j] = variance

    texture_map = cv2.normalize(texture_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return texture_map.astype(np.uint8)





def iterative_otsu_for_texture_maps(feature_maps, iterations_list):
    """
    Apply iterative Otsu's thresholding to each feature map using a distinct number of iterations.
    
    Parameters:
        feature_maps (list): List of 2D feature maps.
        iterations_list (list): Number of iterations for each feature map.
        
    Returns:
        refined_masks (list): List of refined binary masks after iterative Otsu.
    """
    refined_masks = []
    
    for feature_map, iterations in zip(feature_maps, iterations_list):
        initial_threshold, initial_mask = otsu_threshold(feature_map)
        refined_mask = initial_mask
        
        # Perform the iterative refinement
        for _ in range(iterations):
            refined_mask = refine_foreground_segmentation(feature_map, refined_mask, initial_threshold)
        
        refined_masks.append(refined_mask)
    
    return refined_masks

def extract_texture_features(image, window_sizes):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature_maps = [compute_texture_features(grayscale_image, N) for N in window_sizes]
    return feature_maps

def save_texture_based_masks(refined_masks, window_sizes):
    """
    Save the refined masks for each texture-based feature map.
    
    Parameters:
        refined_masks (list): List of refined masks.
        window_sizes (list): List of window sizes corresponding to each mask.
    """
    for idx, mask in enumerate(refined_masks):
        filename = f'dog_refined_mask_window_{window_sizes[idx]}.png'
        cv2.imwrite(filename, mask)
        print(f"Saved {filename}")






In [ ]:
# Example usage:
# Load an RGB image and split into channels
dog_image = cv2.imread('pics/dog_small.jpg')
R, G, B = cv2.split(dog_image)

# Define the number of iterations for each channel
iterations_dict = {'R': 1, 'G': 1, 'B': 1}

# Apply iterative Otsu for each channel
R_mask = iterative_otsu(R, iterations_dict['R']) 
G_mask = iterative_otsu(G, iterations_dict['G'])
B_mask = iterative_otsu(B, iterations_dict['B'])

# Combine the masks using bitwise AND
final_mask = combine_masks_with_bitwise_and([R_mask, G_mask, B_mask])

cv2.imwrite('dog_red_mask.png', R_mask)
cv2.imwrite('dog_green_mask.png', G_mask)
cv2.imwrite('dog_blue_mask.png', B_mask)
cv2.imwrite('dog_refined_mask.png', final_mask)

window_sizes = [3, 5, 9]
iterations_list = [2, 2, 2]  # Number of iterations for each window size feature map

# Step 1: Extract texture-based features
feature_maps = extract_texture_features(dog_image, window_sizes)

refined_masks = iterative_otsu_for_texture_maps(feature_maps, iterations_list)

# Step 3: Save each refined mask
save_texture_based_masks(refined_masks, window_sizes)

# Optional: Combine the masks for a final result
final_mask = combine_masks_with_bitwise_and(refined_masks)
cv2.imwrite('dog_final_texture_based_mask.png', final_mask)


In [152]:
# def compute_texture_features(image, N):
#     """
#     Compute the intensity variance within an N x N window around each pixel.
    
#     Parameters:
#         image (2D numpy array): Grayscale image.
#         N (int): Size of the sliding window (e.g., 3, 5, or 7).
        
#     Returns:
#         texture_map (2D numpy array): Feature map with intensity variance.
#     """
#     # Step 1: Define the window size
#     pad = N // 2  # Padding size for border handling
    
#     # Step 2: Pad the image to handle border pixels (assuming 0 outside border)
#     padded_image = np.pad(image, pad, mode='constant', constant_values=0)
    
#     # Step 3: Initialize the texture map with zeros
#     texture_map = np.zeros_like(image, dtype=np.float32)
    
#     # Step 4: Compute the texture feature (variance) using the sliding window
#     for i in range(image.shape[0]):
#         for j in range(image.shape[1]):
#             # Extract the N x N window centered at (i, j)
#             window = padded_image[i:i + N, j:j + N]
            
#             # Calculate the mean intensity of the window
#             mean_intensity = np.mean(window)
            
#             # Calculate the variance of the window as the texture measure
#             variance = np.mean((window - mean_intensity) ** 2)
            
#             # Store the variance in the texture map at the center pixel
#             texture_map[i, j] = variance
    
#     # Normalize the texture map to the range [0, 255]
#     texture_map = cv2.normalize(texture_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
#     return texture_map.astype(np.uint8)

# def apply_iterative_otsu_on_feature_maps(feature_maps):
#     """
#     Apply iterative Otsu's thresholding to each feature map to refine results.
    
#     Parameters:
#         feature_maps (list): List of 2D feature maps.
        
#     Returns:
#         refined_masks (list): List of refined binary masks after iterative Otsu.
#     """
#     refined_masks = []
    
#     for feature_map in feature_maps:
#         # Step 1: Apply initial Otsu's thresholding
#         initial_threshold, initial_mask = otsu_threshold(feature_map)
        
#         # Step 2: Refine the mask using the foreground pixels
#         refined_mask = refine_foreground_segmentation(feature_map, initial_mask, initial_threshold)
        
#         refined_masks.append(refined_mask)
    
#     return refined_masks

# def extract_texture_features(image, window_sizes):
#     """
#     Extract texture-based features for different window sizes.
    
#     Parameters:
#         image (3D numpy array): RGB image.
#         window_sizes (list): List of window sizes (e.g., [3, 5, 7]).
        
#     Returns:
#         feature_maps (list): List of texture maps for each window size.
#     """
#     # Convert the image to grayscale
#     grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Extract texture maps for each window size
#     feature_maps = [compute_texture_features(grayscale_image, N) for N in window_sizes]
#     return feature_maps

# def apply_otsu_on_feature_maps(feature_maps):
#     """
#     Apply Otsu's thresholding to each feature map.
    
#     Parameters:
#         feature_maps (list): List of 2D feature maps.
        
#     Returns:
#         thresholds (list): Optimal thresholds for each feature map.
#         masks (list): Binary masks after applying Otsu's thresholding on each map.
#     """
#     thresholds = []
#     masks = []
    
#     for feature_map in feature_maps:
#         # Apply Otsu's thresholding
#         optimal_threshold, thresh = otsu_threshold(feature_map)
#         thresholds.append(optimal_threshold)
#         masks.append(thresh)
    
#     return thresholds, masks





In [153]:
# def refine_foreground_segmentation_iterative(image, initial_thresh, max_iterations=10, tolerance=1):
#     """
#     Iteratively refines the foreground segmentation using Otsu's algorithm.
    
#     Parameters:
#         image (2D numpy array): Grayscale image.
#         initial_thresh (2D numpy array): Initial binary thresholded image.
#         max_iterations (int): Maximum number of iterations for refinement.
#         tolerance (int): Number of pixels allowed to differ before considering convergence.
        
#     Returns:
#         refined_mask (2D numpy array): Refined binary mask after iterative Otsu.
#     """
#     # Step 1: Initialize with the initial threshold and mask
#     current_mask = initial_thresh
#     previous_mask = np.zeros_like(image, dtype=np.uint8)
    
#     # Step 2: Run iterative refinement
#     for i in range(max_iterations):
#         # Extract foreground pixels using the current mask
#         foreground_pixels = image[current_mask == 255]
        
#         # Check if there are enough foreground pixels to refine further
#         if len(foreground_pixels) == 0:
#             break  # No more foreground pixels to refine

#         # Apply Otsu's threshold on the foreground pixels to get a refined threshold
#         refined_threshold = otsu_threshold(foreground_pixels)[0]
        
#         # Create a new mask based on the refined threshold
#         refined_mask = np.zeros_like(image, dtype=np.uint8)
#         refined_mask[(image > refined_threshold) & (current_mask == 255)] = 255
        
#         # Check for convergence: if the mask changes very little, stop iterating
#         pixel_difference = np.sum(np.abs(refined_mask - current_mask))
#         if pixel_difference <= tolerance:
#             print(f"Converged after {i + 1} iterations with pixel difference: {pixel_difference}")
#             break
        
#         # Update masks for the next iteration
#         previous_mask = current_mask
#         current_mask = refined_mask
    
#     return current_mask

# # Function to combine masks using logical AND
# def combine_masks_with_bitwise_and(masks):
#     """
#     Combine binary masks using a logical AND operation.
    
#     Parameters:
#         masks (list): List of binary masks (2D arrays).
        
#     Returns:
#         combined_mask (2D numpy array): Combined binary mask.
#     """
#     combined_mask = masks[0]
    
#     # Apply bitwise AND with subsequent masks
#     for mask in masks[1:]:
#         combined_mask = np.bitwise_and(combined_mask, mask)
    
#     return combined_mask

# # Function to find optimal threshold using Otsu's method
# def otsu_threshold(image):
#     """
#     Computes Otsu's threshold and returns the threshold value and binary mask.
    
#     Parameters:
#         image (2D numpy array): Grayscale image.
        
#     Returns:
#         optimal_threshold (int): Otsu's threshold value.
#         thresh (2D numpy array): Binary mask after thresholding.
#     """
#     # Compute histogram
#     hist = compute_histogram(image)
    
#     # Normalize the histogram
#     total_pixels = image.size
#     prob_hist = normalize_histogram(hist, total_pixels)
    
#     # Compute cumulative sums and means
#     cum_sum, cum_mean = compute_cumulative_sums_and_means(prob_hist)
    
#     # Compute global mean
#     global_mean = cum_mean[-1]
    
#     # Find optimal threshold using between-class variance maximization
#     max_variance = -1
#     optimal_threshold = 0
    
#     for t in range(256):
#         # Background and foreground probabilities
#         w0 = cum_sum[t]
#         w1 = 1 - w0
        
#         if w0 == 0 or w1 == 0:
#             continue  # Avoid division by zero
        
#         # Means of the two classes
#         mean0 = cum_mean[t] / w0
#         mean1 = (cum_mean[-1] - cum_mean[t]) / w1
        
#         # Calculate between-class variance
#         between_class_variance = w0 * w1 * (mean0 - mean1) ** 2
        
#         # Update optimal threshold
#         if between_class_variance > max_variance:
#             max_variance = between_class_variance
#             optimal_threshold = t
    
#     # Threshold the image using the optimal threshold
#     thresh = np.where(image > optimal_threshold, 255, 0).astype(np.uint8)
    
#     return optimal_threshold, thresh

# # Example usage:
# # Load a grayscale image
# image = cv2.imread('path_to_grayscale_image.jpg', cv2.IMREAD_GRAYSCALE)

# # Step 1: Apply initial Otsu's threshold
# initial_threshold, initial_mask = otsu_threshold(image)

# # Step 2: Refine the segmentation iteratively
# refined_mask = refine_foreground_segmentation_iterative(image, initial_mask)

# # Save the final refined mask as an image
# cv2.imwrite('final_refined_mask_iterative.png', refined_mask)

# print("Refined Otsu mask saved as 'final_refined_mask_iterative.png'.")


In [154]:


# def bitwise_and(mask1, mask2):
#     # Initialize the result with the same shape as the input masks
#     result = np.zeros_like(mask1)
    
#     # Perform bitwise AND: set the result to 255 (white) where both mask1 and mask2 are 255 (white)
#     result[(mask1 == 255) & (mask2 == 255)] = 255
    
#     return result




# # Function to refine foreground segmentation
# def refine_foreground_segmentation(image, thresh, optimal_threshold):
#     # Step 1: Extract the preliminary foreground using the initial threshold
#     foreground_pixels = image[thresh == 255]
    
#     # Step 2: Apply Otsu's threshold to the foreground pixels to refine the thresholding
#     if len(foreground_pixels) > 0:
#         refined_threshold = otsu_threshold(foreground_pixels)[0]
#     else:
#         refined_threshold = optimal_threshold  # Use the original threshold if no foreground is detected
    
#     # Step 3: Create the final refined mask
#     # Use the refined threshold on the original image, but only where the initial foreground was detected
#     refined_mask = np.zeros_like(image, dtype=np.uint8)
#     refined_mask[(image > refined_threshold) & (thresh == 255)] = 255
    
#     return refined_mask

# def combine_masks_with_bitwise_and(masks):
#     """
#     Combine binary masks using a logical AND operation.
    
#     Parameters:
#         masks (list): List of binary masks (2D arrays).
        
#     Returns:
#         combined_mask (2D numpy array): Combined binary mask.
#     """
#     # Start with the first mask
#     combined_mask = masks[0]
    
#     # Apply bitwise AND with subsequent masks
#     for mask in masks[1:]:
#         combined_mask = np.bitwise_and(combined_mask, mask)
    
#     return combined_mask

# # Function to find optimal threshold using Otsu's method
# def otsu_threshold(image):
#     # Step 1: Compute histogram
#     hist = compute_histogram(image)
    
#     # Step 2: Normalize the histogram
#     total_pixels = image.size
#     prob_hist = normalize_histogram(hist, total_pixels)
    
#     # Step 3: Compute cumulative sums and means
#     cum_sum, cum_mean = compute_cumulative_sums_and_means(prob_hist)
    
#     # Step 4: Compute global mean
#     global_mean = cum_mean[-1]
    
#     # Step 5: Initialize variables to find optimal threshold
#     max_variance = -1
#     optimal_threshold = 0
    
#     for t in range(256):
#         # Probabilities of two classes (background and foreground)
#         w0 = cum_sum[t]  # Background class
#         w1 = 1 - w0      # Foreground class
        
#         if w0 == 0 or w1 == 0:
#             continue  # Avoid division by zero
        
#         # Means of the two classes
#         mean0 = cum_mean[t] / w0
#         mean1 = (cum_mean[-1] - cum_mean[t]) / w1
        
#         # Step 6: Compute between-class variance
#         between_class_variance = w0 * w1 * (mean0 - mean1) ** 2
        
#         # Step 7: Update the optimal threshold if the variance is maximal
#         if between_class_variance > max_variance:
#             max_variance = between_class_variance
#             optimal_threshold = t
    
#     # Step 8: Threshold the image using the optimal threshold
#     thresh = np.where(image > optimal_threshold, 255, 0).astype(np.uint8)
    
#     return optimal_threshold, thresh


# # Load the RGB image

# import numpy as np
# import cv2


In [155]:
# # Function to compute variance within an NxN window at each pixel
# def compute_texture_features(image, N):
#     # Get the dimensions of the image
#     rows, cols = image.shape
    
#     # Create a new feature image to store variance values
#     texture_feature_map = np.zeros((rows, cols), dtype=np.float32)
    
#     # Pad the image to handle the borders (assuming 0 outside the image)
#     pad_size = N // 2
#     padded_image = np.pad(image, pad_size, mode='constant', constant_values=0)

#     # Compute variance for each pixel
#     for i in range(pad_size, rows + pad_size):
#         for j in range(pad_size, cols + pad_size):
#             # Extract the N×N window
#             window = padded_image[i-pad_size:i+pad_size+1, j-pad_size:j+pad_size+1]
            
#             # Compute mean and variance within the window
#             mean_intensity = np.mean(window)
#             variance = np.var(window)
            
#             # Store the variance as a texture feature for the current pixel
#             texture_feature_map[i-pad_size, j-pad_size] = variance

#     return texture_feature_map

# # Function to compute variance-based texture feature map
# def compute_texture_variance(image, N):
#     # Padding the image to handle borders
#     pad_size = N // 2
#     padded_image = np.pad(image, pad_size, mode='constant', constant_values=0)
#     variance_map = np.zeros_like(image, dtype=np.float32)
    
#     # Compute variance in N x N window
#     for i in range(image.shape[0]):
#         for j in range(image.shape[1]):
#             window = padded_image[i:i + N, j:j + N]
#             mean = np.mean(window)
#             variance = np.mean((window - mean) ** 2)
#             variance_map[i, j] = variance
    
#     return variance_map











In [156]:
# image = cv2.imread('pics/dog_small.jpg')


# B, G, R = cv2.split(image)
# # Apply the custom Otsu thresholding function to the Red channel
# optimal_threshold_R, R_thresh = otsu_threshold(R)
# optimal_threshold_G, G_thresh = otsu_threshold(G)
# optimal_threshold_B, B_thresh = otsu_threshold(B)
# refined_foreground_R = refine_foreground_segmentation(R, R_thresh, optimal_threshold_R)
# refined_foreground_G = refine_foreground_segmentation(G, G_thresh, optimal_threshold_G)
# refined_foreground_B = refine_foreground_segmentation(B, B_thresh, optimal_threshold_B)
# final_mask = combine_masks_with_bitwise_and([refined_foreground_R, refined_foreground_B, refined_foreground_G])
# cv2.imwrite('dog_refined_mask_R.png', refined_foreground_R)
# cv2.imwrite('dog_refined_mask_G.png', refined_foreground_G)
# cv2.imwrite('dog_refined_mask_B.png', refined_foreground_B)
# cv2.imwrite('dog_combined_mask.png', final_mask)

# # Step 5: Define window sizes to try (e.g., 3, 5, 7)
# window_sizes = [3, 5, 7]

# # Step 6: Extract texture-based features using sliding window approach
# feature_maps = extract_texture_features(image, window_sizes)

# # Step 7: Apply Otsu's algorithm to each feature map
# thresholds, masks = apply_otsu_on_feature_maps(feature_maps)

# # Step 8: Combine the resulting masks using bitwise AND
# final_mask = combine_masks_with_bitwise_and(masks)

# # Save the final combined mask as an image
# cv2.imwrite('dog_text_final_mask.png', final_mask)



# # Refining
# refined_masks = apply_iterative_otsu_on_feature_maps(feature_maps)

# # Combine the refined masks using bitwise AND
# final_mask = combine_masks_with_bitwise_and(refined_masks)

# # Save the final combined mask as an image
# cv2.imwrite('dog_text_final_refined_mask.png', final_mask)